# 数据加载与预处理
使用Hugging Face的datasets库加载NarrativeQA

In [ ]:
import datasets
import pandas as pd
from typing import List, Dict, Tuple


class NarrativeQALoader:
    def __init__(self, split: str = "test"):
        """
        初始化加载器
        Args:
            split: 'train', 'validation', 或 'test'
        """
        print(f"正在加载 NarrativeQA 数据集 ({split} split)...")
        # 直接从Hugging Face Hub加载
        from pyprojroot import here
        dataset_path = here() / "dataset"
        self.dataset = datasets.load_dataset("narrativeqa", split=split, cache_dir=dataset_path)

    def get_data(self) -> Tuple[List[str], List[List[str]], List[str]]:
        """
        提取评估所需的关键字段
        Returns:
            questions: 问题列表
            references: 参考答案列表（每个元素为包含两个答案的列表）
            doc_ids: 文档ID列表，用于后续关联原文
        """
        questions = []
        references = []
        doc_ids = []

        for item in self.dataset:
            # 提取问题文本
            q_text = item['question']['text']

            # 提取参考答案
            # NarrativeQA的结构中，answers是一个包含两个对象的列表
            # 每个对象有 'text' 字段
            ans_texts = [ans['text'] for ans in item['answers']]

            # 提取文档ID
            d_id = item['document']['id']

            questions.append(q_text)
            references.append(ans_texts)
            doc_ids.append(d_id)

        return questions, references, doc_ids

    def get_corpus(self) -> Dict[str, str]:
        """
        提取文档全文（如果需要进行检索实验）
        Returns:
            corpus_dict: {doc_id: full_text}
        """
        corpus_dict = {}
        # 注意：NarrativeQA的全文在 document.text 字段
        # 部分样本可能只包含摘要，需检查 kind 字段
        for item in self.dataset:
            doc_id = item['document']['id']
            # 这里简化处理，直接取text字段。实际可能需要清洗Gutenberg headers
            text = item['document']['text']
            corpus_dict[doc_id] = text
        return corpus_dict

加载数据集，如果本地没有数据集会下载

数据被分为三组：

| train | validation | test  |
| ----- | ----- | ----- |
| 32747 | 3461  | 10557 |

In [ ]:
loader = NarrativeQALoader(split="validation")

测试数据集

In [ ]:
qs, refs, ids = loader.get_data()
print(f"加载完成，共 {len(qs)} 个测试样本。")
print(f"示例问题: {qs}")
print(f"示例参考答案: {refs}")

In [ ]:
one = loader.dataset[0]

In [ ]:
print(one)
print(one.keys())  # ['document', 'question', 'answers']
print(one.get('document').keys())
print(one.get('document').get('summary'))
print(one.get('document').get('id'))  # 获取文档id
print(one.get('document').get('text'))  # 获取文档全文

# 评测

In [ ]:
import string
import numpy as np
import evaluate
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer


class NarrativeQAEvaluator:
    def __init__(self):
        # 初始化 ROUGE Scorer
        self.rouge_scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        # 初始化 METEOR (通过Hugging Face evaluate加载)
        self.meteor_metric = evaluate.load('meteor')
        # BLEU平滑函数，用于处理短文本
        self.smoothing = SmoothingFunction().method1

    def normalize_answer(self, s: str) -> str:
        """
        标准化答案文本：
        1. 小写
        2. 去除标点
        3. 规范化空白字符
        这是复现官方分数的关键步骤。
        """

        def white_space_fix(text):
            return ' '.join(text.split())

        def remove_punc(text):
            exclude = set(string.punctuation)
            return ''.join(ch for ch in text if ch not in exclude)

        def lower(text):
            return text.lower()

        return white_space_fix(remove_punc(lower(s)))

    def evaluate_one_sample(self, prediction: str, references: List[str]) -> Dict[str, float]:
        """
        对单个样本计算所有指标
        """
        # 1. 文本标准化
        pred_norm = self.normalize_answer(prediction)
        refs_norm = [self.normalize_answer(r) for r in references]

        # 2. Tokenization (用于BLEU)
        pred_tokens = word_tokenize(pred_norm)
        refs_tokens = [word_tokenize(r) for r in refs_norm]

        metrics = {}

        # --- ROUGE-L (Max over references) ---
        # 对每个参考答案分别计算ROUGE-L，取F-measure最大值
        rouge_scores = [self.rouge_scorer.score(r, pred_norm)['rougeL'].fmeasure for r in refs_norm]
        metrics = max(rouge_scores) if rouge_scores else 0.0

        # --- BLEU-1 & BLEU-2 (Max over references) ---
        # 尽管sentence_bleu支持多个references输入列表，但为了严格遵循Max策略，
        # 且为了与部分文献对齐，这里显式对每对计算后取最大。
        # 另一种常见做法是直接传入refs_tokens列表作为reference corpus，这会计算corpus level micro-average。
        # 针对NarrativeQA，DeepMind官方脚本计算的是 micro-averaged 还是 max? 
        # 官方脚本通常使用 mteval-v13a.pl，它是计算 corpus level 的。
        # 但在Python复现中，对每个样本取最大值也是一种稳健的 approximated sentence-level 方法。
        # 下面演示 Sentence-Level Max 策略：

        b1_scores = [sentence_bleu([ref], pred_tokens, weights=(1, 0, 0, 0), smoothing_function=self.smoothing) for ref
                     in refs_tokens]
        metrics = max(b1_scores) if b1_scores else 0.0

        b2_scores = [sentence_bleu([ref], pred_tokens, weights=(0.5, 0.5, 0, 0), smoothing_function=self.smoothing) for
                     ref in refs_tokens]
        metrics = max(b2_scores) if b2_scores else 0.0

        # --- METEOR ---
        # METEOR 本身设计支持多参考答案，内部会自动处理对齐。
        # 我们可以直接将所有references传给metric。
        # 注意：evaluate库的compute方法通常是batch处理，这里为了逻辑清晰单条调用，
        # 实际生产中应批量调用以提高速度。
        # 由于evaluate接口限制，稍后在batch_evaluate中统一处理METEOR。

        return metrics

    def batch_evaluate(self, predictions: List[str], references: List[List[str]]) -> Dict[str, float]:
        """
        批量评估整个数据集
        """
        total_scores = {'ROUGE-L': 0, 'BLEU-1': 0, 'BLEU-2': 0, 'METEOR': 0}

        # 预处理数据用于METEOR批量计算
        clean_preds = [self.normalize_answer(p) for p in predictions]
        clean_refs = [[self.normalize_answer(r) for r in ref_list] for ref_list in references]

        # 批量计算 METEOR (利用evaluate库的优化)
        print("正在计算 METEOR...")
        meteor_res = self.meteor_metric.compute(predictions=clean_preds, references=clean_refs)
        # 这一步返回的是corpus level的平均分，还是每个sentence的分数？
        # evaluate的meteor返回 {'meteor': float}，是整个语料的平均分。
        # 这与ROUGE/BLEU的sentence-level average略有不同，但在报告中作为系统级得分为标准做法。
        total_scores = meteor_res['meteor']

        # 逐个计算 ROUGE 和 BLEU
        print("正在计算 ROUGE 和 BLEU...")
        for i, (p, r) in enumerate(zip(predictions, references)):
            res = self.evaluate_one_sample(p, r)
            total_scores.append(res)
            total_scores.append(res)
            total_scores.append(res)

        # 汇总结果
        final_metrics = {
            'ROUGE-L': np.mean(total_scores),
            'BLEU-1': np.mean(total_scores),
            'BLEU-2': np.mean(total_scores),
            'METEOR': total_scores
        }

        return final_metrics

# RAG系统与主程序

In [ ]:
class SimpleRAGSystem:
    def __init__(self):
        # 初始化检索器和LLM
        # e.g., self.retriever = Chroma(...)
        # e.g., self.llm = OpenAI(...)
        pass

    def generate(self, question: str, doc_id: str) -> str:
        """
        模拟生成过程。
        此处仅为了演示代码跑通，返回一个随机的占位符答案。
        实际应为：
        1. chunks = self.retriever.query(question, filter={doc_id: doc_id})
        2. prompt = build_prompt(chunks, question)
        3. answer = self.llm(prompt)
        """
        return "This is a generated answer based on the retrieved context."


# 主执行脚本
if __name__ == "__main__":
    # 1. 准备数据
    loader = NarrativeQALoader(split="test")
    questions, references, doc_ids = loader.get_data()

    # 缩小数据集用于快速测试 (例如前100条)
    SAMPLE_SIZE = 100
    questions = questions
    references = references
    doc_ids = doc_ids

    # 2. 运行RAG系统生成预测
    rag_system = SimpleRAGSystem()
    predictions =

    print(f"开始生成预测 (共 {len(questions)} 条)...")
    for q, d_id in zip(questions, doc_ids):
        # 实际生成
        # pred = rag_system.generate(q, d_id)

        # 演示用：为了让分数不为0，我们随机选取一个参考答案并添加噪声
        import random

        mock_pred = references[questions.index(q)] + " extra words"
        predictions.append(mock_pred)

    # 3. 运行评估
    evaluator = NarrativeQAEvaluator()
    results = evaluator.batch_evaluate(predictions, references)

    # 4. 输出最终报告
    print("\n" + "=" * 40)
    print("NarrativeQA RAG 系统测评结果报告")
    print("=" * 40)
    # 使用Markdown表格格式输出
    print(f"| Metric | Score |")
    print(f"| :--- | :--- |")
    print(f"| ROUGE-L | {results:.4f} |")
    print(f"| BLEU-1 | {results:.4f} |")
    print(f"| BLEU-2 | {results:.4f} |")
    print(f"| METEOR | {results:.4f} |")
    print("=" * 40)